In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from pmdarima import auto_arima
from prophet import Prophet

In [ ]:
# Import Data
us_diff = pd.read_csv('univariate_datasets/us_diff.csv', usecols=[1,2], index_col=0, parse_dates=[0])
us = pd.read_csv('univariate_datasets/univariate_df_United_States.csv',usecols=[1,2], index_col=0, parse_dates=[0])
belgium = pd.read_csv('univariate_datasets/univariate_df_Belgium.csv', usecols=[1,2], index_col = 0, parse_dates = [0])
chile = pd.read_csv('univariate_datasets/univariate_df_Chile.csv', usecols=[1,2], index_col = 0, parse_dates = [0])
czechia = pd.read_csv('univariate_datasets/univariate_df_Czechia.csv', usecols=[1,2], index_col = 0, parse_dates = [0])
estonia = pd.read_csv('univariate_datasets/univariate_df_Estonia.csv', usecols=[1,2], index_col = 0, parse_dates = [0])
france = pd.read_csv('univariate_datasets/univariate_df_France.csv', usecols=[1,2], index_col = 0, parse_dates = [0])
ireland = pd.read_csv('univariate_datasets/univariate_df_Ireland.csv', usecols=[1,2], index_col = 0, parse_dates = [0])
israel = pd.read_csv('univariate_datasets/univariate_df_Israel.csv', usecols=[1,2], index_col = 0, parse_dates = [0])
italy = pd.read_csv('univariate_datasets/univariate_df_Italy.csv', usecols=[1,2], index_col = 0, parse_dates = [0])
malaysia = pd.read_csv('univariate_datasets/univariate_df_Malaysia.csv', usecols=[1,2], index_col = 0, parse_dates = [0])
# test_train split
train_size = int(len(us)*0.8)

small_constant = 1
def mape(y_true, prediction):
    return np.mean(np.abs((y_true - prediction) / (y_true + small_constant))) * 100

evaluation = pd.DataFrame(columns = ['Country','Model', 'MAE', 'MAPE'])